In [1]:
from pyunicorn.timeseries.surrogates import Surrogates
import pyunicorn
import numpy as np
from scipy.stats import norm
from netCDF4 import Dataset
import random
import csv
import PCA_functions as pf
import pandas as pd
from Data import Data
from datetime import datetime
import math
import pandas as pd

In [18]:
class Surrogates:

    """
    Encapsulates structures and methods related to surrogate time series.
    Provides data structures and methods to generate surrogate data sets from a
    set of time series and to evaluate the significance of various correlation
    measures using these surrogates.
    More information on time series surrogates can be found in [Schreiber2000]_
    and [Kantz2006]_.
    """

    #
    #  Define internal methods
    #
    def __init__(self, original_data, silence_level=1):
        """
        Initialize an instance of Surrogates.
        .. note::
           The order of array dimensions is different from the standard of
           ``core``. Here it is [index, time] for reasons of computational
           speed!
        :type original_data: 2D array [index, time]
        :arg original_data: The original time series for surrogate generation.
        :arg int silence_level: The inverse level of verbosity of the object.
        """
        if silence_level <= 1:
            print("Generated an instance of the Surrogates class.")

        #  Set class variables
        self.original_data = original_data
        """The original time series for surrogate generation."""
        self.silence_level = silence_level
        """(string) - The inverse level of verbosity of the object."""

        #  Set flags
        self._normalized = False
        self._fft_cached = False
        self._twins_cached = False

        #  Cache
        self._twins = None
        self._original_data_fft = None
        
        

    def clear_cache(self):
        """Clean up cache."""
        try:
            del self._original_data_fft
            del self._twins
        except AttributeError:
            pass
    def correlated_noise_surrogates(self, original_data):
        """
        Return Fourier surrogates.
        Generate surrogates by Fourier transforming the :attr:`original_data`
        time series (assumed to be real valued), randomizing the phases and
        then applying an inverse Fourier transform. Correlated noise surrogates
        share their power spectrum and autocorrelation function with the
        original_data time series.
        The Fast Fourier transforms of all time series are cached to facilitate
        a faster generation of several surrogates for each time series. Hence,
        :meth:`clear_cache` has to be called before generating surrogates from
        a different set of time series!
        .. note::
           The amplitudes are not adjusted here, i.e., the
           individual amplitude distributions are not conserved!
        **Examples:**
        The power spectrum is conserved up to small numerical deviations:
        >>> ts = Surrogates.SmallTestData().original_data
        >>> surrogates = Surrogates.\
                SmallTestData().correlated_noise_surrogates(ts)
        >>> all(r(np.abs(np.fft.fft(ts,         axis=1))[0,1:10]) == \
                r(np.abs(np.fft.fft(surrogates, axis=1))[0,1:10]))
        True
        However, the time series amplitude distributions differ:
        >>> all(np.histogram(ts[0,:])[0] == np.histogram(surrogates[0,:])[0])
        False
        :type original_data: 2D array [index, time]
        :arg original_data: The original time series.
        :rtype: 2D array [index, time]
        :return: The surrogate time series.
        """
        if self.silence_level <= 1:
            print("Generating correlated noise surrogates...")

        #  Calculate FFT of original_data time series
        #  The FFT of the original_data data has to be calculated only once,
        #  so it is stored in self._original_data_fft.
        if self._fft_cached:
            surrogates = self._original_data_fft
        else:
            surrogates = np.fft.rfft(original_data, axis=1)
            self._original_data_fft = surrogates
            self._fft_cached = True

        #  Get shapes
        (N, n_time) = original_data.shape
        len_phase = surrogates.shape[1]

        #  Generate random phases uniformly distributed in the
        #  interval [0, 2*Pi]
        phases = np.random.uniform(low=0, high=2 * np.pi, size=(N, len_phase))

        #  Add random phases uniformly distributed in the interval [0, 2*Pi]
        surrogates *= np.exp(1j * phases)

        #  Calculate IFFT and take the real part, the remaining imaginary part
        #  is due to numerical errors.
        return np.ascontiguousarray(np.real(np.fft.irfft(surrogates, n=n_time,
                                                         axis=1)))

In [19]:
def deseasonalize(data,freq=12):
    """
    The shape of data should be (time, index) 
    """
    n  = data.shape[1]
    N  = data.shape[0]
    data_deseasonal = np.zeros(data.shape)
    for i in range(n):
        temp = np.copy(data[:,i])
        r = np.zeros((N))
        for j in range(freq):
            Idx = np.arange(j,N,freq)
            if temp[Idx].std() == 0:
                r[Idx] = 0
            else:
                r[Idx] = (temp[Idx] - temp[Idx].mean())/temp[Idx].std()
        data_deseasonal[:,i] = np.copy(r)
    return(data_deseasonal)

In [20]:
level = 12
temporal_limits = {"time_min":datetime(1948, 1, 1, 0, 0),"time_max":datetime(2016, 1, 1, 0, 0) }

In [5]:
sst = Data('../nc/sst.mnmean.nc','sst',temporal_limits, missing_value= -9.96921e+36)

result = sst.get_data()
lon_sst_list = sst.get_lon_list()
lat_sst_list = sst.get_lat_list()
lon_sst = sst.get_lon()
lat_sst = sst.get_lat()

data = deseasonalize(np.array(result))
weights = np.sqrt(np.abs(np.cos(np.array(lat_sst_list)* math.pi/180)))
for i in range(len(weights)):
    data[:,i] = weights[i] * data[:,i]

In [21]:
d = Data('../nc/sst.mnmean.nc','sst',temporal_limits, missing_value= -9.96921e+36)

result = d.get_data()
lon_list = d.get_lon_list()
lat_list = d.get_lat_list()
lon = d.get_lon()
lat = d.get_lat()

result = deseasonalize(np.array(result))
data = np.transpose(result)
weights = np.sqrt(np.abs(np.cos(np.array(lat_list)* math.pi/180)))

for i in range(len(weights)):
    data[i,:] = weights[i] * data[i,:]

p_value = 0.05/data.shape[0]

In [27]:
data.shape

(10988, 817)

In [ ]:
eigenValues, eigenVectors = np.linalg.eig(np.cov(data))
idx = eigenValues.argsort()[::-1]
eigenValues = eigenValues[idx]

In [37]:
result.shape[0]

356

In [36]:
result.iloc[:,0].values.mean()

565.4282440271938

In [24]:
result = pd.read_csv("limitTemp_sst.csv")

In [12]:
result.iloc[1,:].values.mean()

0.6760628857148552

In [5]:
data1.shape[0]

356

In [ ]:


    N = 200
    M = data.shape[0]
    result = np.zeros((N,M))
    for i in range(N):
        ts = Surrogates(data)
        sample = ts.correlated_noise_surrogates(data)
        eigenValues, eigenVectors = np.linalg.eig(np.cov(sample))
        idx = eigenValues.argsort()[::-1]
        eigenValues = eigenValues[idx]
        result[i,:] = eigenValues
        ts.clear_cache()
        with open('limitTemp_sst.csv','a') as f:
            writer = csv.writer(f)
            writer.writerow(result[i,:])



    limit = np.zeros(N)
    for i in range(N):
        #limit[i] = 1 - norm.cdf((eigenValues[i] - result[i,:].mean())/result[i,:].std())
        limit[i] = 1 - norm.cdf((eigenValues[i] - result[:,i].mean())/(result[:,i].std()/math.sqrt(N)))
        if limit[i] > p_value: break

    f = open("limit_sst.txt", "a")
    f.write("{} is {}\n".format(name[j], i))


In [21]:
data = np.load('data.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'data.npy'

In [4]:
data = np.load('spi12.npy')

In [22]:
f_pre = Dataset('../nc/precipitation.nc')
data = f_pre.variables['precip']
lon = f_pre.variables['lon'][:]
lat = f_pre.variables['lat'][:]
time = f_pre.variables['time'][:]
data = np.swapaxes(data,0,2)

In [11]:
result = []

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if not data[i,j,-1]<0:
        #if not np.isnan(data[i,j,-1]):   
            result.append(data[i,j,11:])


In [12]:
result = pf.deseasonalize(np.transpose(np.array(result)))

In [13]:
data = np.matrix(result)
data = np.transpose(data)

In [5]:
f_pre = Dataset('GPCC_half.nc')
data = f_pre.variables['precip']
data = np.swapaxes(data,0,2)

#data = np.load('data.npy')

result = []

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if not data[i,j,-1]<0:
        #if not np.isnan(data[i,j,-1]):   
            result.append(data[i,j,:])


result = pf.deseasonalize(np.transpose(np.array(result)))
data = np.matrix(result)
data = np.transpose(data)

In [6]:
result = pd.read_csv("limitTemp_half.csv")

In [8]:
result = np.matrix(result)

In [9]:
result.shape

(129, 13434)

In [4]:
level = 12
temporal_limits = {"time_min":datetime(1948, 1, 1, 0, 0),"time_max":datetime(2016, 1, 1, 0, 0) }

In [50]:
name = ["precipitation.nc","pres.mon.mean.nc","precipitation.nc"]
id = ["precip", "pres", "precip"]
missing = [-9.969209968386869e+36, -9.969209968386869e+36, -9.969209968386869e+36]

In [52]:
for j in range(len(name)):
    d = Data('../nc/{}'.format(name[j]),code[j],temporal_limits,missing_value=missing[j])

    result = d.get_data()
    lon_list = d.get_lon_list()
    lat_list = d.get_lat_list()
    lon = d.get_lon()
    lat = d.get_lat()
    
    result = pf.deseasonalize(np.array(result))
    data = np.transpose(result)
    weights = np.sqrt(np.cos(np.array(lat_list)* math.pi/180))

    for i in range(len(weights)):
        data[i,:] = weights[i] * data[i,:]
        
    N = 20
    M = data.shape[0]
    result = np.zeros((N,M))

    for i in range(N):
        ts = Surrogates(data)
        #sample = ts.correlated_noise_surrogates(data)
        sample = ts.refined_AAFT_surrogates(data, n_iterations=100)

        eigenValues, eigenVectors = np.linalg.eig(np.corrcoef(sample))
        idx = eigenValues.argsort()[::-1]   
        eigenValues = eigenValues[idx]
        result[i,:] = eigenValues
        ts.clear_cache()
        
    eigenValues, eigenVectors = np.linalg.eig(np.corrcoef(data))
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    
    limit = np.zeros(M)
    for i in range(M):
        limit[i] = 1 - norm.cdf((eigenValues[i] - result[:,i].mean())/result[:,i].std())
        if limit[i] > p_value: break
    f = open("limit.txt", "a")
    f.write("{} is {}\n".format(name[j], i))
    f.close()

Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance

# Number of Components os SPI

In [67]:
level = 12
temporal_limits = {"time_min":datetime(1946, 1, 1, 0, 0),"time_max":datetime(2016, 1, 1, 0, 0) }
spatial_limits = {"lon_min":-40,"lon_max":60,"lat_min":-40,"lat_max":40}


In [68]:
d = Data('../nc/precip.mon.total.2.5x2.5.v2018.nc','precip',temporal_limits,missing_value=-9.96921e+36)

result = d.get_data()
lon_list = d.get_lon_list()
lat_list = d.get_lat_list()
lon = d.get_lon()
lat = d.get_lat()


In [77]:
result = np.load("World_gamma.npy")
result = result[:-1,:]
index = np.where(np.isinf(result))
result[index] = -2

In [78]:
result.shape

(468, 3638)

In [79]:
np.save("World_gamma.npy",result)

In [80]:
result = np.load("World_gamma.npy")
data = np.transpose(result[:-1,:])
index = np.where(np.isinf(data))
data[index] = -2.3263478740408408

In [81]:
weights = np.sqrt(np.abs(np.cos(np.array(lat_list)* math.pi/180)))

In [82]:
for i in range(len(weights)):
    data[i,:] = weights[i] * data[i,:]

In [83]:
eigenValues, eigenVectors = np.linalg.eig(np.corrcoef(data))
idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]

In [84]:
np.count_nonzero(eigenValues > (1+math.sqrt(data.shape[0]/data.shape[1]))**2)

46

In [5]:
d = Data('../nc/sst.mnmean.nc','sst',temporal_limits,missing_value=-9.96921e+36)

result = d.get_data()
lon_list = d.get_lon_list()
lat_list = d.get_lat_list()
lon = d.get_lon()
lat = d.get_lat()

In [92]:
result  = result[:,3:]

In [103]:
c = np.corrcoef(np.transpose(result))

/home/mathsys2/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/mathsys2/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [104]:
c.shape

(10988, 10988)

In [31]:
np.count_nonzero(np.isnan(c))

5038275

In [107]:
c = np.nan_to_num(c)

In [82]:
c

array([[ 1.        ,  0.29036095,  0.13682917, ..., -0.04993022,
        -0.04357997, -0.03129445],
       [ 0.29036095,  1.        ,  0.5300719 , ..., -0.26086539,
        -0.22697259, -0.15930152],
       [ 0.13682917,  0.5300719 ,  1.        , ..., -0.58991362,
        -0.52411689, -0.40143879],
       ...,
       [-0.04993022, -0.26086539, -0.58991362, ...,  1.        ,
         0.96663335,  0.78364142],
       [-0.04357997, -0.22697259, -0.52411689, ...,  0.96663335,
         1.        ,  0.82862391],
       [-0.03129445, -0.15930152, -0.40143879, ...,  0.78364142,
         0.82862391,  1.        ]])

In [84]:
result = np.transpose(result)

In [48]:
lat_list[0]

88.0

In [85]:
result  = result[3:,:]

In [86]:
lat_list = lat_list[3:]

In [87]:
lon_list = lon_list[3:]

In [9]:
data.shape

(10988, 817)

In [6]:
result = pf.deseasonalize(np.array(result))

In [7]:
data = np.transpose(result)

In [8]:
weights = np.sqrt(np.abs(np.cos(np.array(lat_list)* math.pi/180)))

for i in range(len(weights)):
    data[i,:] = weights[i] * data[i,:]

In [33]:
(1+math.sqrt(data.shape[0]/data.shape[1]))**2

3.540451048526307

In [34]:
eigenValues, eigenVectors = np.linalg.eig(np.corrcoef(data))
idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]

In [35]:
np.count_nonzero(eigenValues > (1+math.sqrt(data.shape[0]/data.shape[1]))**2)

39

In [37]:
N = 20
M = data.shape[0]
result = np.zeros((N,M))

for i in range(N):
    ts = Surrogates(data)
    sample = ts.correlated_noise_surrogates(data)
    #sample = ts.AAFT_surrogates(data)
    
    eigenValues, eigenVectors = np.linalg.eig(np.corrcoef(sample))
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    result[i,:] = eigenValues
    ts.clear_cache()
    #with open('limit.csv','a') as f:
     #   writer = csv.writer(f)
      #  writer.writerow(result[i,:])

Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance

In [38]:
eigenValues, eigenVectors = np.linalg.eig(np.corrcoef(data))
idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]

In [39]:
p_value = 0.05/data.shape[0]

In [40]:
p_value

7.874015748031497e-05

In [43]:
limit = np.zeros(M)
for i in range(M):
    limit[i] = 1 - norm.cdf((eigenValues[i] - result[:,i].mean())/(result[:,i].std()/math.sqrt(N)))
    #print("%f"%(1 - norm.cdf((eigenValues[i] - result[:,i].mean())/result[:,i].std())))
    if limit[i] > p_value: break

In [44]:
i

59

In [41]:
from numpy import random

In [ ]:
random.ra

In [ ]:
#from pyunicorn.pyunicorn.timeseries.surrogates import Surrogates
import numpy as np
from scipy.stats import norm
from netCDF4 import Dataset
import random
import csv
#import PCA_functions as pf
import pandas as pd
from Data import Data
from datetime import datetime
import math

def deseasonalize(data,freq=12):
    """
    The shape of data should be (time, index) 
    """
    n  = data.shape[1]
    N  = data.shape[0]
    data_deseasonal = np.zeros(data.shape)
    for i in range(n):
        temp = np.copy(data[:,i])
        r = np.zeros((N))
        for j in range(freq):
            Idx = np.arange(j,N,freq)
            if temp[Idx].std() == 0:
                r[Idx] = 0
            else:
                r[Idx] = (temp[Idx] - temp[Idx].mean())/temp[Idx].std()
        data_deseasonal[:,i] = np.copy(r)
    return(data_deseasonal)


level = 12
temporal_limits = {"time_min":datetime(1948, 1, 1, 0, 0),"time_max":datetime(2016, 1, 1, 0, 0) }

name = ["precip.mon.total.2.5x2.5.v2018.nc","pres.mon.mean.nc","air.mon.mean.nc","sst.mnmean.nc"]
code = ["precip", "pres", "air", "sst"]
missing = [-9.96921e+36, -9.96921e+36, -9.96921e+36,-9.96921e+36]

for j in range(len(name)):
    d = Data('{}'.format(name[j]),code[j],temporal_limits,missing_value=missing[j])
    
    result = d.get_data()

    result = deseasonalize(np.array(result))
    data = np.transpose(result)
    weights = np.sqrt(np.cos(np.array(lat_list)* math.pi/180))

    eigenValues, eigenVectors = np.linalg.eig(np.corrcoef(data))
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]

    number = np.count_nonzero(eigenValues > (1+math.sqrt(data.shape[0]/data.shape[1]))**2)

    f = open("number.txt", "a")
    f.write("{} is {}\n".format(name[j], number))
    f.close()


In [27]:
#from pyunicorn.pyunicorn.timeseries.surrogates import Surrogates
import numpy as np
from scipy.stats import norm
from netCDF4 import Dataset
import random
import csv
#import PCA_functions as pf
import pandas as pd
from Data import Data
from datetime import datetime
import math

def deseasonalize(data,freq=12):
    """
    The shape of data should be (time, index) 
    """
    n  = data.shape[1]
    N  = data.shape[0]
    data_deseasonal = np.zeros(data.shape)
    for i in range(n):
        temp = np.copy(data[:,i])
        r = np.zeros((N))
        for j in range(freq):
            Idx = np.arange(j,N,freq)
            if temp[Idx].std() == 0:
                r[Idx] = 0
            else:
                r[Idx] = (temp[Idx] - temp[Idx].mean())/temp[Idx].std()
        data_deseasonal[:,i] = np.copy(r)
    return(data_deseasonal)

In [109]:
level = 12
temporal_limits = {"time_min":datetime(1948, 1, 1, 0, 0),"time_max":datetime(2016, 1, 1, 0, 0) }

name = "../nc/sst.mnmean.nc"
code =  "sst"
missing = -9.96921e+36

In [110]:
d = Data('{}'.format(name),code,temporal_limits,missing_value=missing)

result = d.get_data()
lon_list = d.get_lon_list()
lat_list = d.get_lat_list()
lon = d.get_lon()
lat = d.get_lat()

In [112]:
result = pf.deseasonalize(np.array(result))
data = np.transpose(result)
#weights = np.sqrt(np.cos(np.array(lat_list)* math.pi/180))
weights = np.sqrt(np.abs(np.cos(np.array(lat_list)* math.pi/180)))

In [113]:
for i in range(len(weights)):
    data[i,:] = weights[i] * data[i,:]

In [114]:
data.shape

(10988, 817)

In [115]:
c = np.corrcoef(data)

/home/mathsys2/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/mathsys2/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [116]:
c.shape

(10988, 10988)

In [117]:
np.count_nonzero(np.isnan(c))

8270119

In [ ]:




    eigenValues, eigenVectors = np.linalg.eig(np.corrcoef(data))
    idx = eigenValues.argsort()[::-1]
    eigenValues = eigenValues[idx]

    number = np.count_nonzero(eigenValues > (1+math.sqrt(data.shape[0]/data.shape[1]))**2)

    f = open("number.txt", "a")
    f.write("{} is {}\n".format(name[j], number))
                                                   

/home/mathsys2/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/mathsys2/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [26]:
c

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ..., 1.        , 0.73245799,
        0.21527619],
       [       nan,        nan,        nan, ..., 0.73245799, 1.        ,
        0.22548907],
       [       nan,        nan,        nan, ..., 0.21527619, 0.22548907,
        1.        ]])

In [18]:
data.shape

(635, 817)

In [3]:
limit = np.load("limit.npy")

In [42]:
for i in range(M):
    if limit[i]<0.05 and limit[i+1]>0.05:
        print("%f"%(i))
        break

58.000000


In [3]:
for i in range(2000):
    print("%f"%(limit[i]))

0.000000
0.000000
0.000000
0.000080
0.013419
0.027787
0.048526
0.103424
0.135305
0.172163
0.178612
0.199780
0.221764
0.229446
0.237565
0.245592
0.253328
0.258030
0.269907
0.281925
0.286005
0.289908
0.295204
0.298934
0.314354
0.321130
0.323891
0.328486
0.328781
0.334500
0.338656
0.344893
0.347458
0.350352
0.351458
0.355565
0.357270
0.363188
0.366705
0.369545
0.371359
0.374384
0.378778
0.383102
0.385895
0.388062
0.389705
0.389904
0.392658
0.394333
0.396781
0.397980
0.400858
0.403030
0.404643
0.405562
0.406200
0.407440
0.408048
0.411537
0.412496
0.413446
0.414504
0.415819
0.418182
0.418624
0.420086
0.420962
0.423300
0.424828
0.427017
0.427347
0.428506
0.429176
0.429790
0.430453
0.431078
0.433078
0.433578
0.434989
0.435408
0.435998
0.436965
0.437502
0.438386
0.438917
0.439860
0.440453
0.440903
0.442092
0.442602
0.442978
0.444101
0.445441
0.445816
0.446626
0.447967
0.448344
0.448730
0.450147
0.450623
0.450881
0.451247
0.451804
0.452514
0.453862
0.453986
0.454386
0.455190
0.455322
0.455529
0

In [2]:
limit = np.load("limit.npy")

In [4]:
l.shape

(2000,)

In [9]:
result = pd.read_csv("limitTemp_sat.csv", header=None)

In [11]:
result.shape[0]

141

In [10]:
result

,0,1,2,3,4,5,6,7,8,9,...,10502,10503,10504,10505,10506,10507,10508,10509,10510,10511
0,296.664705,287.832584,142.288438,139.567812,78.088094,77.483258,64.708525,62.544155,61.806616,60.637433,...,-7.192547e-15,-7.491634e-15,-7.491634e-15,-8.133343e-15,-8.133343e-15,-8.674336e-15,-1.067499e-14,-1.272100e-14,-1.272100e-14,-1.360260e-14
1,294.166047,290.466372,142.248632,139.939101,78.527027,76.355189,65.008875,64.048027,61.737083,58.859806,...,-7.117739e-15,-7.150851e-15,-7.150851e-15,-7.349323e-15,-7.349323e-15,-7.755031e-15,-8.617684e-15,-8.617684e-15,-1.248835e-14,-1.248835e-14
2,292.660334,291.928162,144.294377,137.935038,78.410441,77.062274,63.840711,62.837632,61.301065,61.196068,...,-7.270502e-15,-7.351398e-15,-7.351398e-15,-7.898357e-15,-7.898357e-15,-8.408286e-15,-8.605321e-15,-1.696933e-14,-1.696933e-14,-2.031696e-14
3,301.697323,282.974790,141.440911,140.870948,77.740926,76.759670,64.986200,63.397551,62.412103,58.875047,...,-7.226109e-15,-7.226109e-15,-7.458789e-15,-8.064397e-15,-8.064397e-15,-1.066540e-14,-1.066540e-14,-1.100563e-14,-1.612762e-14,-1.612762e-14
4,298.882496,285.558326,142.449980,139.672385,79.158038,76.219813,63.575604,63.496865,62.154323,60.344694,...,-7.411842e-15,-7.411842e-15,-7.714167e-15,-7.714167e-15,-8.490333e-15,-8.725034e-15,-8.725034e-15,-1.074620e-14,-1.074620e-14,-1.315635e-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,299.990109,284.498040,143.127272,139.009193,78.984268,76.164044,64.615217,63.355589,61.575155,60.431857,...,-7.180987e-15,-7.180987e-15,-7.258230e-15,-7.556901e-15,-7.556901e-15,-8.094959e-15,-8.094959e-15,-1.124137e-14,-1.258163e-14,-1.258163e-14
137,295.406615,289.751641,144.785361,136.543918,79.338580,75.819234,64.982155,63.579136,61.876858,59.328644,...,-7.453994e-15,-7.634056e-15,-7.634056e-15,-7.835373e-15,-7.835373e-15,-8.770826e-15,-9.727780e-15,-9.727780e-15,-1.257819e-14,-1.257819e-14
138,295.967576,288.463789,143.426674,138.430404,78.386937,76.341398,64.793472,62.646928,62.011448,60.342353,...,-7.352379e-15,-7.446160e-15,-7.921612e-15,-7.921612e-15,-8.335324e-15,-8.335324e-15,-8.659648e-15,-8.659648e-15,-1.203804e-14,-1.761986e-14
139,298.167306,286.573298,143.322915,138.648306,78.025303,77.040845,64.819083,63.766968,60.690450,60.424742,...,-7.062153e-15,-7.170505e-15,-7.221975e-15,-7.221975e-15,-7.824251e-15,-7.824251e-15,-8.980298e-15,-8.980298e-15,-1.199299e-14,-1.199299e-14


In [ ]:
M = result.shape[1]
limit = np.zeros(M)
for i in range(M):
    limit[i] = 1 - norm.cdf((eigenValues[i] - result.iloc[:,i].mean())/result.iloc[:,i].std())
    #print("%f"%(1 - norm.cdf((eigenValues[i] - result[:,i].mean())/result[:,i].std())))
    if limit[i] > p_value: break

In [34]:
d = np.row_stack((a,b,c))

In [48]:
ts = Surrogates(d)

Generated an instance of the Surrogates class.


In [47]:
ts.AAFT_surrogates(d)

Generating correlated noise surrogates...


AttributeError: 'Surrogates' object has no attribute '_original_data_fft'

Generating correlated noise surrogates...


array([[ 3.9791028 ,  5.45287519,  5.02532696,  1.94243231,  1.3774935 ,
         3.02007958,  1.86420421,  3.7646391 ],
       [ 4.22527408,  2.90988095,  7.94998781,  3.50909721, -2.71409865,
         4.7119573 ,  4.94455026,  4.76008565],
       [-3.59516812, -1.97702414, -6.70089651, -6.96161177, -6.30666783,
        -5.75830119, -4.50530105, -3.28342285]])

In [39]:
ts.white_noise_surrogates(d)

Generating white noise surrogates by random shuffling...


array([[ 3,  5,  5,  1,  3,  5,  4,  2],
       [ 8, 12,  7,  6,  5,  3,  3,  4],
       [ 2,  7,  6,  7,  5,  4,  6,  3]])

In [45]:
ts.refined_AAFT_surrogates(d,1000)

AttributeError: 'Surrogates' object has no attribute '_original_data_fft'

In [58]:
N = 10
result = np.zeros((N,3))
for i in range(N):
    ts = Surrogates(d)
    sample = ts.correlated_noise_surrogates(d)
    eigenValues, eigenVectors = np.linalg.eig(np.cov(sample))
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    result[i,:] = eigenValues
    ts.clear_cache()

Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...
Generated an instance of the Surrogates class.
Generating correlated noise surrogates...


In [50]:
np.cov(d)

array([[ 2.28571429, -1.28571429,  2.71428571],
       [-1.28571429,  9.14285714, -1.        ],
       [ 2.71428571, -1.        ,  3.42857143]])

In [51]:
d

array([[ 1,  2,  3,  3,  4,  5,  5,  5],
       [12,  3,  4,  5,  6,  7,  3,  8],
       [ 2,  3,  4,  5,  6,  7,  6,  7]])

In [60]:
eigenValues, eigenVectors = np.linalg.eig(np.cov(d))
idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]

In [59]:
result

array([[ 9.90562985,  3.93204551,  0.82269823],
       [10.91110084,  2.33278413,  1.31345101],
       [11.46662559,  2.34593331,  0.78591232],
       [10.25746858,  3.63521738,  0.62321524],
       [11.58200185,  2.7306186 ,  0.15794229],
       [ 9.92805697,  3.51538064,  1.12691524],
       [10.02253596,  3.37299933,  1.38606517],
       [ 9.21170952,  4.57287297,  0.99065836],
       [10.01374339,  4.10176874,  0.3919988 ],
       [10.86831356,  3.15128156,  0.55702729]])

In [61]:
eigenValues

array([9.76570372, 5.02381354, 0.06762559])